<a href="https://colab.research.google.com/github/Untick/InspectrumClinic_RS_gr2/blob/main/Borisov%20Vasily/Profpatolog.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import gdown
base_dir = '/content/PrPt'
gdown.download('https://storage.yandexcloud.net/terratraineeship/23_InspectrumClinic_RS/datasets/датасет%20профпатолог%20часть.xlsx', None, quiet=True)
data = pd.read_excel('датасет%20профпатолог%20часть.xlsx')

In [37]:
df_polei = pd.read_excel('df_polei.xlsx')
data.shape
#arrZakMK = data.ЗаключениеМК.dropna().unique()
arrZakMK = ['Годен', 'ГоденСКоррекциейЗрения', 'ГоденБезРаботНаВысотах', 'НуждаетсяВДообследованииИЛечении', 'ВременноНегоден', 'ОграниченноГоден']
data.shape


(1982, 4587)

In [38]:
data500  = data.copy() 
nmp=df_polei['POLE'].tolist()
print(nmp)


for (columnName, columnData) in data500.items():
  if (str(columnName) in nmp)==False:
    data500.pop(columnName)

data500 = data500[data500.ЗаключениеМК.isin(arrZakMK) == True ]
print(data500.shape)
  

['КлиентПолПредставление', 'Профессия', 'ЗаключениеМК', 'ВредныеФакторы', 'Рентгенология1_Заключение_ЗначениеПредставление', 'ПсихиатрияНаркология1_МКБ101', 'ПсихиатрияНаркология1_ГоденНегоден_ЗначениеПредставление', 'Офтальмология1_МКБ101', 'Офтальмология1_ГоденНегоден_ЗначениеПредставление', 'Оториноларингология1_МКБ101', 'Оториноларингология1_ГоденНегоден_ЗначениеПредставление', 'ХирургНеИсп1_МКБ101', 'ХирургНеИсп1_ГоденНегоден_ЗначениеПредставление', 'Гинекология1_МКБ101', 'Гинекология1_ГоденНегоден_ЗначениеПредставление', 'Стоматология1_МКБ101', 'Стоматология1_ГоденНегоден_ЗначениеПредставление', 'Терапия1_ГоденНегоден_ЗначениеПредставление', 'Дерматовенерология1_МКБ101', 'Дерматовенерология1_ГоденНегоден_ЗначениеПредставление', 'Рентгенология1_МКБ101', 'Рентгенология1_ПрофнепригодностьВременная1', 'Неврология1_МКБ101', 'Неврология1_ПрофнепригодностьВременная1', 'Неврология1_ГоденНегоден_ЗначениеПредставление', 'Терапия1_МКБ101', 'Хирургия1_МКБ101', 'Хирургия1_ГоденНегоден_Значени

In [40]:
nparrColumnNames = []
nparrColumnValues = []
column_list = []
for (columnName, columnData) in data500.items():
  if data500[columnName].value_counts().shape[0]<20:
    nparrColumnNames.append(columnName)
    nparrColumnValues.append(list(data500[columnName].dropna().unique().flatten()))


In [ ]:
data500.to_excel("data500.xlsx")

,ЗаключениеМК,ЗаключениеМКПредставление,ДиспансернаяГруппа,ПсихиатрияНаркология1_ПрофнепригодностьВременнаяПредставление1,ПсихиатрияНаркология1_ДатаПриема_Значение,ПсихиатрияНаркология1_Врач_Значение,ПсихиатрияНаркология1_ГоденНегоден_ЗначениеПредставление,ПсихиатрияНаркология1_Жалобы_Значение,ПсихиатрияНаркология1_Анамнез_Значение,Офтальмология1_ПрофнепригодностьВременнаяПредставление1,...,Рентгенология1_ПрофнепригодностьВременнаяПредставление2,Рентгенология2_ПрофнепригодностьВременнаяПредставление2,Психиатрия3_ПрофнепригодностьВременнаяПредставление1,Психиатрия3_ПрофнепригодностьВременнаяПредставление2,Психиатрия3_ГоденНегоден_ЗначениеПредставление,Психиатрия4_ПрофнепригодностьВременнаяПредставление1,Психиатрия4_ПрофнепригодностьВременнаяПредставление2,Психиатрия4_ГоденНегоден_ЗначениеПредставление,Оториноларингология3_ПрофнепригодностьВременнаяПредставление1,Оториноларингология3_ГоденНегоден_ЗначениеПредставление
0,Годен,Годен к работе в указанных условиях,I,NaN,25.01.2022,Батырев М.Я.,Годен,1b66edfc-243b-11e8-80c9-0cc47aab8067,3d58fa6c-243b-11e8-80c9-0cc47aab8067,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Годен,Годен к работе в указанных условиях,IIIа,Годен,02.02.2022,Батырев М.Я.,Годен,1b66edfc-243b-11e8-80c9-0cc47aab8067,3d58fa6c-243b-11e8-80c9-0cc47aab8067,Годен,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,ГоденСКоррекциейЗрения,Годен к работе в указанных условиях (с коррекц...,II,Годен,15.02.2022,Батырев М.Я.,Годен,1b66edfc-243b-11e8-80c9-0cc47aab8067,3d58fa6c-243b-11e8-80c9-0cc47aab8067,Годен,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Годен,Годен к работе в указанных условиях,I,NaN,NaN,NaN,NaN,NaN,NaN,Годен,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,ГоденСКоррекциейЗрения,Годен к работе в указанных условиях (с коррекц...,II,NaN,NaN,NaN,NaN,NaN,NaN,Годен,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


(1571, 143)
(1571, 143)
(1571, 6)


In [ ]:
from tensorflow.keras.models import Sequential 
from tensorflow.keras.layers import Dense 
from tensorflow.keras.optimizers import Adam 
from tensorflow.keras import utils 
import matplotlib.pyplot as plt 
from PIL import Image 
%matplotlib inline 

